In [3]:
# import required libraries
# pandas:Data framework library for Python
# sklearn: Library to perform machine learning tasks
import pandas as pd
import numpy as np
import scipy
import scipy.stats as st
import sklearn 
import sklearn.datasets
import sklearn.metrics as metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.feature_selection import mutual_info_classif
from scipy import stats

In [5]:
Data_2017 = pd.read_excel("PERM_Disclosure_Data_FY17.xlsx")

In [8]:
Data_2015 = pd.read_excel("PERM_Disclosure_Data_FY15_Q4.xlsx")

In [14]:
Data_2016 = pd.read_excel("PERM_Data_FY16.xlsx")

In [60]:
nm15 = list(Data_2015.columns.values)
nm16 = list(Data_2016.columns.values)
nm17 = list(Data_2017.columns.values)

In [36]:
#Change the column names so they match between years
Data_2016=Data_2016.rename(columns = {'FW_INFO_REL_OCCUP_EXPERIENCE':'FW_INFO_REL_OCCUP_EXP'})
Data_2015=Data_2015.rename(columns = {'JI_FOREIGN_WORKER_LIVE_ON_PREMISES':'JI_FW_LIVE_ON_PREMISES'})
Data_2015=Data_2015.rename(columns = {'REC_INFO_BARG_REP_NOTIFIED':'RECR_INFO_BARG_REP_NOTIFIED'})
Data_2015=Data_2015.rename(columns = {'JI_OFFERED_TO_SEC_J_FOREIGN_WORKER':'JI_OFFERED_TO_SEC_J_FW'})
Data_2017=Data_2017.rename(columns = {'PW_JOB_TITLE_9089':'PW_Job_Title_9089'})

In [65]:
frames = [Data_2015, Data_2016, Data_2016]

data = pd.concat(frames)

In [77]:
data.shape
#number of rows, cols

(341585, 125)

In [71]:
data.CASE_STATUS.unique()

array(['Certified-Expired', 'Certified', 'Denied', 'Withdrawn'], dtype=object)

In [86]:
(data['CASE_STATUS'].value_counts() / data['CASE_STATUS'].value_counts().sum()) * 100

Certified            46.466619
Certified-Expired    44.522154
Denied                5.011637
Withdrawn             3.999590
Name: CASE_STATUS, dtype: float64

In [87]:
(data['REFILE'].value_counts() / data['REFILE'].value_counts().sum()) * 100
#small percentage of refile, no need

N    99.857999
Y     0.142001
Name: REFILE, dtype: float64

In [88]:
(data['SCHD_A_SHEEPHERDER'].value_counts() / data['SCHD_A_SHEEPHERDER'].value_counts().sum()) * 100

N    100.0
Name: SCHD_A_SHEEPHERDER, dtype: float64

In [92]:
data['EMPLOYER_NUM_EMPLOYEES'].count()

341462

In [93]:
(data['FW_OWNERSHIP_INTEREST'].value_counts() / data['FW_OWNERSHIP_INTEREST'].value_counts().sum()) * 100

N    99.511522
Y     0.488478
Name: FW_OWNERSHIP_INTEREST, dtype: float64

In [95]:
data['AGENT_FIRM_NAME'].isnull().sum() / 341585

0.13221013803299325

In [96]:
(data['JI_OFFERED_TO_SEC_J_FW'].value_counts() / data['JI_OFFERED_TO_SEC_J_FW'].value_counts().sum()) * 100

Y    99.98624
N     0.01376
Name: JI_OFFERED_TO_SEC_J_FW, dtype: float64

In [97]:
(data['JI_FW_LIVE_ON_PREMISES'].value_counts() / data['JI_FW_LIVE_ON_PREMISES'].value_counts().sum()) * 100

N    99.863742
Y     0.136258
Name: JI_FW_LIVE_ON_PREMISES, dtype: float64

In [98]:
(data['JI_LIVE_IN_DOMESTIC_SERVICE'].value_counts() / data['JI_LIVE_IN_DOMESTIC_SERVICE'].value_counts().sum()) * 100

N    99.923519
Y     0.076481
Name: JI_LIVE_IN_DOMESTIC_SERVICE, dtype: float64

In [99]:
(data['JI_LIVE_IN_DOM_SVC_CONTRACT'].value_counts() / data['JI_LIVE_IN_DOM_SVC_CONTRACT'].value_counts().sum()) * 100

A    85.752979
Y     7.123510
N     7.123510
Name: JI_LIVE_IN_DOM_SVC_CONTRACT, dtype: float64

In [ ]:
RI_LAYOFF_IN_PAST_SIX_MONTHS

In [100]:
(data['RECR_INFO_EMPLOYER_REC_PAYMENT'].value_counts() / data['RECR_INFO_EMPLOYER_REC_PAYMENT'].value_counts().sum()) * 100

N    99.993266
Y     0.006734
Name: RECR_INFO_EMPLOYER_REC_PAYMENT, dtype: float64

In [101]:
(data['RI_LAYOFF_IN_PAST_SIX_MONTHS'].value_counts() / data['RI_LAYOFF_IN_PAST_SIX_MONTHS'].value_counts().sum()) * 100

N    95.91722
Y     4.08278
Name: RI_LAYOFF_IN_PAST_SIX_MONTHS, dtype: float64

In [102]:
(data['RI_US_WORKERS_CONSIDERED'].value_counts() / data['RI_US_WORKERS_CONSIDERED'].value_counts().sum()) * 100

Y    50.050251
A    48.840356
N     1.109393
Name: RI_US_WORKERS_CONSIDERED, dtype: float64

In [103]:
(data['RI_COLL_TCH_BASIC_PROCESS'].value_counts() / data['RI_COLL_TCH_BASIC_PROCESS'].value_counts().sum()) * 100

N    88.816934
Y    11.183066
Name: RI_COLL_TCH_BASIC_PROCESS, dtype: float64

In [104]:
(data['RECR_INFO_COLL_TEACH_COMP_PROC'].value_counts() / data['RECR_INFO_COLL_TEACH_COMP_PROC'].value_counts().sum()) * 100

Y    81.268707
N    18.731293
Name: RECR_INFO_COLL_TEACH_COMP_PROC, dtype: float64

In [105]:
recr_dates = data[["RECR_INFO_SWA_JOB_ORDER_START","RECR_INFO_SWA_JOB_ORDER_END","RECR_INFO_FIRST_AD_START","RECR_INFO_SECOND_AD_START","RECR_INFO_JOB_FAIR_FROM","RECR_INFO_JOB_FAIR_TO","RECR_INFO_ON_CAMPUS_RECR_FROM","RECR_INFO_ON_CAMPUS_RECR_TO","RI_EMPLOYER_WEB_POST_FROM","RI_EMPLOYER_WEB_POST_TO","RECR_INFO_PRO_ORG_ADVERT_FROM","RECR_INFO_PRO_ORG_ADVERT_TO","RI_JOB_SEARCH_WEBSITE_FROM","RI_JOB_SEARCH_WEBSITE_TO","RI_PVT_EMPLOYMENT_FIRM_FROM","RI_PVT_EMPLOYMENT_FIRM_TO","RI_EMPLOYEE_REFERRAL_PROG_FROM","RI_EMPLOYEE_REFERRAL_PROG_TO","RI_CAMPUS_PLACEMENT_FROM","RI_CAMPUS_PLACEMENT_TO","RI_LOCAL_ETHNIC_PAPER_FROM","RI_LOCAL_ETHNIC_PAPER_TO","RECR_INFO_RADIO_TV_AD_FROM","RECR_INFO_RADIO_TV_AD_TO"]]

In [108]:
recr_dates.head(n=10)

,RECR_INFO_SWA_JOB_ORDER_START,RECR_INFO_SWA_JOB_ORDER_END,RECR_INFO_FIRST_AD_START,RECR_INFO_SECOND_AD_START,RECR_INFO_JOB_FAIR_FROM,RECR_INFO_JOB_FAIR_TO,RECR_INFO_ON_CAMPUS_RECR_FROM,RECR_INFO_ON_CAMPUS_RECR_TO,RI_EMPLOYER_WEB_POST_FROM,RI_EMPLOYER_WEB_POST_TO,...,RI_PVT_EMPLOYMENT_FIRM_FROM,RI_PVT_EMPLOYMENT_FIRM_TO,RI_EMPLOYEE_REFERRAL_PROG_FROM,RI_EMPLOYEE_REFERRAL_PROG_TO,RI_CAMPUS_PLACEMENT_FROM,RI_CAMPUS_PLACEMENT_TO,RI_LOCAL_ETHNIC_PAPER_FROM,RI_LOCAL_ETHNIC_PAPER_TO,RECR_INFO_RADIO_TV_AD_FROM,RECR_INFO_RADIO_TV_AD_TO
0,2014-05-15 00:00:00,2014-06-27 00:00:00,2014-05-25,2014-06-01,NaT,NaT,NaT,NaT,2014-05-15 00:00:00,2014-06-27,...,NaT,NaT,NaN,NaT,2014-05-23 00:00:00,2014-06-27,NaN,NaT,NaT,NaT
1,2014-05-29 00:00:00,2014-07-03 00:00:00,2014-06-01,2014-06-08,NaT,NaT,NaT,NaT,2014-05-30 00:00:00,2014-06-27,...,NaT,NaT,NaN,NaT,NaN,NaT,2014-06-04 00:00:00,2014-06-04,NaT,NaT
2,2014-03-10 00:00:00,2014-04-11 00:00:00,2014-03-23,2014-03-30,NaT,NaT,NaT,NaT,NaN,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,2014-03-26 00:00:00,2014-03-26,NaT,NaT
3,2014-06-05 00:00:00,2014-07-07 00:00:00,2014-06-08,2014-06-15,NaT,NaT,NaT,NaT,NaN,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,2014-06-13 00:00:00,2014-06-13,NaT,NaT
4,2014-04-01 00:00:00,2014-05-06 00:00:00,2014-04-27,2014-05-04,NaT,NaT,NaT,NaT,2014-04-22 00:00:00,2014-06-26,...,NaT,NaT,2014-06-16 00:00:00,2014-07-03,NaN,NaT,NaN,NaT,NaT,NaT
5,2014-05-22 00:00:00,2014-06-24 00:00:00,2014-02-02,2014-02-09,NaT,NaT,NaT,NaT,NaN,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,2014-02-03 00:00:00,2014-02-03,NaT,NaT
6,2014-05-17 00:00:00,2014-06-21 00:00:00,2014-06-01,2014-06-08,NaT,NaT,NaT,NaT,2014-05-14 00:00:00,2014-06-17,...,NaT,NaT,2014-05-12 00:00:00,2014-06-16,NaN,NaT,NaN,NaT,NaT,NaT
7,2014-02-20 00:00:00,2014-03-22 00:00:00,2014-06-08,2014-06-15,NaT,NaT,NaT,NaT,2014-06-05 00:00:00,2014-07-09,...,NaT,NaT,NaN,NaT,NaN,NaT,2014-06-08 00:00:00,2014-06-08,NaT,NaT
8,2014-06-13 00:00:00,2014-07-15 00:00:00,2014-06-15,2014-06-22,NaT,NaT,NaT,NaT,2014-06-16 00:00:00,2014-07-16,...,NaT,NaT,NaN,NaT,NaN,NaT,2014-08-21 00:00:00,2014-08-21,NaT,NaT
9,2014-05-15 00:00:00,2014-06-16 00:00:00,2014-06-15,2014-06-22,NaT,NaT,NaT,NaT,NaN,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,2014-06-19 00:00:00,2014-06-19,NaT,NaT


In [129]:
#recr_dates['RECR_INFO_SWA_JOB_ORDER_START'] = pd.to_datetime(recr_dates['RECR_INFO_SWA_JOB_ORDER_START'])
#df['col'] = pd.to_datetime(df['col'])
tmp = recr_dates['RECR_INFO_SWA_JOB_ORDER_START'][126141]
tmp.iloc[0]

datetime.datetime(2015, 11, 2, 0, 0)

In [107]:
recr_dates["RECR_INFO_SWA_JOB_ORDER_END"] - recr_dates["RECR_INFO_SWA_JOB_ORDER_START"]

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'float'